In [3]:
import pandas as pd
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

print( "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ) )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [27]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords):
    review_text = BeautifulSoup(review).get_text()
    review_text=re.sub("[^a-zA-Z]", " ", review_text)
    words = review_text.lower().split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return words
    

In [28]:
import nltk.data
# nltk.download("punkt")
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

In [29]:
def review_to_senteces(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [30]:
sentences = []

for review in train['review']:
    sentences += review_to_senteces(review, tokenizer)

for review in unlabeled_train['review']:
    sentences += review_to_senteces(review, tokenizer)


/tmp/ipykernel_597116/2292428609.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/tmp/ipykernel_597116/2292428609.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


In [32]:
len(sentences)

796172

In [34]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [37]:
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
num_features = 300
min_word_count = 40
num_workers = 8
context = 10
downsampling = 1e-3

from gensim.models import word2vec
print("Starting train model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)
model.init_sims(replace=True)
model_name = "300features_40minwidows_10context"
model.save(model_name)

Starting train model...


TypeError: __init__() got an unexpected keyword argument 'size'